In [20]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

In [21]:
test_csv = pd.read_csv("/Users/shekhar/Desktop/Bootcamp/Revision/Homework/BlackFridayClusting/data/BlackFriday.csv")


In [23]:
#Convert Product Category 2 and 3 into integers
test_csv['Product_Category_2']=test_csv['Product_Category_2'].fillna(0).astype(int)
test_csv['Product_Category_3']=test_csv['Product_Category_3'].fillna(0).astype(int)

#remove Product ID and User ID
data=test_csv.drop(['Product_ID','User_ID'],axis=1)

#label categorical variables
data['Gender']=data['Gender'].map( {'M': 0, 'F': 1} ).astype(int)
data['City_Category']=data['City_Category'].map( {'A': 0, 'B': 1, 'C':2} ).astype(int)
data['Age']=data['Age'].map( {'0-17': 0, '18-25': 1, '26-35': 2,'36-45':3,'46-50':4,
                         '51-55':5,'55+':6} ).astype(int)
data['Stay_In_Current_City_Years']=data['Stay_In_Current_City_Years'].map( {'0': 0, '1': 1, '2': 2,'3':3,'4+':4}).astype(int)

#Get an array of feature variables X and target variable y
X=data.drop(['Purchase'],axis=1).values
y=data['Purchase'].values

#Select features to keep based on percentile of the highest scores
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_regression
Selector_f = SelectPercentile(f_regression, percentile=25)
Selector_f.fit(X,y)

#get the scores of all the features
name_score=list(zip(data.drop(['Purchase'],axis=1).columns.tolist(),Selector_f.scores_))
name_score_df=pd.DataFrame(data=name_score,columns=['Category_names','F_scores'])
name_score_df.sort_values('F_scores',ascending=False)


,Category_names,F_scores
6,Product_Category_1,58852.051269
8,Product_Category_3,47205.918173
3,City_Category,2534.870083
0,Gender,1947.864942
7,Product_Category_2,793.653066
2,Occupation,239.538925
1,Age,168.786478
4,Stay_In_Current_City_Years,16.083005
5,Marital_Status,0.008948


In [27]:
data=test_csv.copy()
data=data[['City_Category','Product_Category_1', 
       'Product_Category_3','Purchase']]
#One-Hot Encoding
data=pd.get_dummies(data=data,columns=['City_Category','Product_Category_1','Product_Category_3'])
data.columns
#Avoid dummy variable trap by removing one category of each categorical feature after encoding but before training
#data.drop(['City_Category_A','Product_Category_1_1','Product_Category_3_0'],axis=1,inplace=True)

X=data.drop(['Purchase'],axis=1).values
y=data['Purchase'].values

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)

from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(X_train,y_train)

y_pred=regressor.predict(X_test)
print("Prediction\n",y_pred)
print("Actual\n",y_test)

print("R_squared Score:",regressor.score(X_test,y_test))



/Users/shekhar/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/shekhar/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/shekhar/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Prediction
 [13289.17399264 16161.65836764 15779.67399264 ...  7368.69743014
  6119.22086764 16215.37711764]
Actual
 [11394 12781 20068 ...  7957  8770 20487]
R_squared Score: 0.6376050662493462
